In [80]:
#!pip install checklist

In [81]:
#!pip install simpletransformers

In [82]:
import numpy as np
import checklist
import pandas as pd
import spacy
from checklist.editor import Editor
from checklist.perturb import Perturb
from simpletransformers.classification import ClassificationModel,ClassificationArgs
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix,average_precision_score

In [83]:
data = pd.read_csv('/content/data/olid-subset-diagnostic-tests.csv')
data.head(3)

,id,text,labels
0,89200,@USER @USER Who the hell does he think he is?,1
1,71294,#BREAKING. #Greece: Molotov cocktails fly afte...,1
2,55633,"#OrrinHatch I can’t believe this sexist , clue...",1


In [84]:
nlp = spacy.load('en_core_web_sm')

In [85]:
perturbed_data = list(nlp.pipe(data['text']))

In [86]:
perturbed_data[:3]

[@USER @USER Who the hell does he think he is?,
 #BREAKING. #Greece: Molotov cocktails fly after protest honouring killed antifa arti... URL via @USER URL,
 #OrrinHatch I can’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!]

In [87]:
ret = Perturb.perturb(data['text'], Perturb.add_typos)
ret.data[:3][1]

['#BREAKING. #Greece: Molotov cocktails fly after protest honouring killed antifa arti... URL via @USER URL',
 '#BREAKING. #Greece: Molotov cocktails lfy after protest honouring killed antifa arti... URL via @USER URL']

In [88]:
typo_data = []
for i in range(len(ret.data)):
  typo_data.append(ret.data[i][1]) # tweet with typos added

typo_data[:3]

['@USER @USER Who the hell does he think h eis?',
 '#BREAKING. #Greece: Molotov cocktails lfy after protest honouring killed antifa arti... URL via @USER URL',
 '#OrrinHatch I cna’t believe this sexist , clueless, old fart gets to weigh in on another woman’s charges against a Supreme Court nominee. And he is spouting the same old nasty shit he spewed 20+ years ago against Anita Hill. His time’s up! Good riddance Neanderthal!']

In [89]:
data_typo = pd.DataFrame({'text':typo_data})
data_typo.head()

,text
0,@USER @USER Who the hell does he think h eis?
1,#BREAKING. #Greece: Molotov cocktails lfy afte...
2,"#OrrinHatch I cna’t believe this sexist , clue..."
3,@USER @USER I'll use that one the next time im...
4,0-1 lost my acca on the first fcuking fight cba


In [90]:
model_args = ClassificationArgs()
model_args.num_train_epochs = 1
model_args.overwrite_output_dir= True
model_args.output_dir = '/content/drive/MyDrive/NLP/error_analyse_model'


model = ClassificationModel("bert", 'bert-base-cased', args=model_args,use_cuda=False)

Some weights of the model checkpoint at bert-base-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model checkpoint at b

In [91]:
train_data = data[['text','labels']]
model.train_model(train_data)

  0%|          | 0/100 [00:00<?, ?it/s]

/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:309: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  FutureWarning,


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Running Epoch 0 of 1:   0%|          | 0/13 [00:00<?, ?it/s]

(13, 0.6986603415929354)

In [92]:
predictions, raw_outputs = model.predict(list(data_typo['text']))


  0%|          | 0/100 [00:00<?, ?it/s]

  0%|          | 0/13 [00:00<?, ?it/s]

In [93]:
predicted_typo_labels = pd.DataFrame({'text':data_typo.text, 'labels': predictions})
predicted_typo_labels.head()

,text,labels
0,@USER @USER Who the hell does he think h eis?,1
1,#BREAKING. #Greece: Molotov cocktails lfy afte...,0
2,"#OrrinHatch I cna’t believe this sexist , clue...",1
3,@USER @USER I'll use that one the next time im...,1
4,0-1 lost my acca on the first fcuking fight cba,1


In [94]:
result, model_outputs, wrong_predictions = model.eval_model(predicted_typo_labels, report = classification_report)

  0%|          | 0/100 [00:00<?, ?it/s]

Running Evaluation:   0%|          | 0/13 [00:00<?, ?it/s]

In [95]:
print(result['report'])

              precision    recall  f1-score   support

         0.0       1.00      1.00      1.00        33
         1.0       1.00      1.00      1.00        67

    accuracy                           1.00       100
   macro avg       1.00      1.00      1.00       100
weighted avg       1.00      1.00      1.00       100



In [96]:
print(model_outputs[:5])

[[-0.67061532 -0.23858932]
 [-0.32077825 -0.37710556]
 [-0.46237618 -0.14922763]
 [-0.65570009 -0.2303382 ]
 [-0.54513955 -0.3187336 ]]


In [97]:
print("Confusion Matrix : ")
print(confusion_matrix(predicted_typo_labels['labels'], predictions))

Confusion Matrix : 
[[33  0]
 [ 0 67]]
